In [2]:
import os
import pandas as pd
import numpy as np
# import dataframe as df
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchmetrics
from torchmetrics.functional import accuracy

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [34]:
X_train = pd.read_csv('../data2.csv')

In [35]:
X_train.head()

,time,id,group,state,HEOL,HEOR,FP1,FP2,VEOU,VEOL,...,P4,T6,A2,O1,OZ,O2,FT9,FT10,PO1,PO2
0,0,0,0,Fatigue,1.215492e-13,-4.869703e-17,-6.140989e-14,2.598510e-17,-1.160435e-13,-3.167843e-17,...,1.651714e-13,-3.307430e-17,3.455894e-13,-6.391263e-17,8.724439e-14,-3.155565e-17,-7.326835e-14,-3.155565e-17,4.404571e-14,-2.665396e-17
1,1,1,0,Fatigue,-7.831237e+01,-1.362717e-02,-5.690807e+01,-4.192949e-02,3.212726e+02,-1.397650e-02,...,-2.646920e+02,-6.988248e-03,-2.281396e+02,4.192949e-02,-5.818660e+01,3.494124e-02,1.040445e+02,3.494124e-02,-3.291532e+01,2.795299e-02
2,2,2,0,Fatigue,-1.533161e+02,-2.671759e-02,-1.114424e+02,-8.209756e-02,6.290321e+02,-2.736585e-02,...,-5.183021e+02,-1.368293e-02,-4.459774e+02,8.209756e-02,-1.138533e+02,6.841464e-02,2.037677e+02,6.841464e-02,-6.439023e+01,5.473171e-02
3,3,3,0,Fatigue,-2.219502e+02,-3.877185e-02,-1.614074e+02,-1.188756e-01,9.107824e+02,-3.962519e-02,...,-7.505852e+02,-1.981260e-02,-6.440388e+02,1.188756e-01,-1.646722e+02,9.906298e-02,2.951723e+02,9.906298e-02,-9.309419e+01,7.925038e-02
4,4,4,0,Fatigue,-2.816333e+02,-4.936209e-02,-2.049461e+02,-1.508884e-01,1.155975e+03,-5.029612e-02,...,-9.528851e+02,-2.514806e-02,-8.144362e+02,1.508884e-01,-2.086840e+02,1.257403e-01,3.748789e+02,1.257403e-01,-1.179092e+02,1.005922e-01


In [36]:
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(X_train.state)

label_encoder.classes_

array(['Fatigue', 'Normal'], dtype=object)

In [37]:
X_train['label'] = encoded_labels
X_train.head()

,time,id,group,state,HEOL,HEOR,FP1,FP2,VEOU,VEOL,...,T6,A2,O1,OZ,O2,FT9,FT10,PO1,PO2,label
0,0,0,0,Fatigue,1.215492e-13,-4.869703e-17,-6.140989e-14,2.598510e-17,-1.160435e-13,-3.167843e-17,...,-3.307430e-17,3.455894e-13,-6.391263e-17,8.724439e-14,-3.155565e-17,-7.326835e-14,-3.155565e-17,4.404571e-14,-2.665396e-17,0
1,1,1,0,Fatigue,-7.831237e+01,-1.362717e-02,-5.690807e+01,-4.192949e-02,3.212726e+02,-1.397650e-02,...,-6.988248e-03,-2.281396e+02,4.192949e-02,-5.818660e+01,3.494124e-02,1.040445e+02,3.494124e-02,-3.291532e+01,2.795299e-02,0
2,2,2,0,Fatigue,-1.533161e+02,-2.671759e-02,-1.114424e+02,-8.209756e-02,6.290321e+02,-2.736585e-02,...,-1.368293e-02,-4.459774e+02,8.209756e-02,-1.138533e+02,6.841464e-02,2.037677e+02,6.841464e-02,-6.439023e+01,5.473171e-02,0
3,3,3,0,Fatigue,-2.219502e+02,-3.877185e-02,-1.614074e+02,-1.188756e-01,9.107824e+02,-3.962519e-02,...,-1.981260e-02,-6.440388e+02,1.188756e-01,-1.646722e+02,9.906298e-02,2.951723e+02,9.906298e-02,-9.309419e+01,7.925038e-02,0
4,4,4,0,Fatigue,-2.816333e+02,-4.936209e-02,-2.049461e+02,-1.508884e-01,1.155975e+03,-5.029612e-02,...,-2.514806e-02,-8.144362e+02,1.508884e-01,-2.086840e+02,1.257403e-01,3.748789e+02,1.257403e-01,-1.179092e+02,1.005922e-01,0


In [5]:
# FEATURE_COLUMNS = X_train.columns.tolist()[4:-2]
FEATURE_COLUMNS = ['TP8', 'FP2', 'FCZ', 'FT10', 'O2', 'O1', 'FT7', 'F4', 'TP7', 'C3', 'C4', 'F3', 'FT8', 'T6', 'HEOR', 'T5', 'VEOL', 'F7', 'FZ', 'VEOU', 'A1', 'P3', 'PZ', 'CP3', 'P4', 'CPZ', 'A2', 'HEOL', 'CP4', 'FT9', 'F8', 'OZ', 'CZ', 'FC4', 'FC3', 'FP1', 'PO1', 'T3', 'T4', 'PO2']
# len(FEATURE_COLUMNS)

40

In [39]:
g = X_train.groupby("group")

In [40]:
sequences = [] 

for name, group in g:
    sequence_features = group[FEATURE_COLUMNS]
    label = group.label.iloc[0]
    
#     print((sequence_features, label))
    sequences.append((sequence_features, label))

In [41]:
sequences[0]

(              HEOL          HEOR           FP1           FP2          VEOU  \
 0     1.215492e-13 -4.869703e-17 -6.140989e-14  2.598510e-17 -1.160435e-13   
 1    -7.831237e+01 -1.362717e-02 -5.690807e+01 -4.192949e-02  3.212726e+02   
 2    -1.533161e+02 -2.671759e-02 -1.114424e+02 -8.209756e-02  6.290321e+02   
 3    -2.219502e+02 -3.877185e-02 -1.614074e+02 -1.188756e-01  9.107824e+02   
 4    -2.816333e+02 -4.936209e-02 -2.049461e+02 -1.508884e-01  1.155975e+03   
 ...            ...           ...           ...           ...           ...   
 1019  9.189533e+00  2.849707e-03  1.783298e+01  6.329993e-03 -4.930598e+01   
 1020  7.436606e+00  2.869100e-03  1.661349e+01  6.348566e-03 -5.123579e+01   
 1021  5.528420e+00  2.921818e-03  1.508811e+01  6.324462e-03 -5.286875e+01   
 1022  3.726376e+00  3.024905e-03  1.349644e+01  6.342777e-03 -5.465339e+01   
 1023  1.962049e+00  3.137497e-03  1.184765e+01  6.318416e-03 -5.574251e+01   
 
               VEOL            F7            F3   

In [42]:
train_sequences, test_sequences = train_test_split(sequences, test_size=0.2)
val_sequences, test_sequences = train_test_split(test_sequences, test_size=0.5)

In [43]:
len(train_sequences), len(val_sequences), len(test_sequences)

(11268, 1409, 1409)

## Data Set

In [44]:
class DrowsyDataset(Dataset):
    
    def __init__(self, sequences):
        self.sequences = sequences
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence, label = self.sequences[idx]
        return dict(
            sequence=torch.Tensor(sequence.to_numpy()),
            label=torch.tensor(label).long()
        )

In [45]:
class DrowsyDataModule(pl.LightningDataModule):
    
    def __init__(self, train_sequences, val_sequences, test_sequences, batch_size):
        super().__init__()
        self.train_sequences = train_sequences
        self.val_sequences = val_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = DrowsyDataset(self.train_sequences)
        self.val_dataset = DrowsyDataset(self.val_sequences)
        self.test_dataset = DrowsyDataset(self.test_sequences)
        
    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=os.cpu_count()
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=os.cpu_count()
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=os.cpu_count()
        ) 

In [46]:
N_EPOCHS = 100
BATCH_SIZE = 50

data_module = DrowsyDataModule(train_sequences, val_sequences, test_sequences, BATCH_SIZE)

## Models

In [47]:
class DrowsyModel(nn.Module):
    
    def __init__(self, n_features, n_classes, n_hidden=256, n_layers=3):
        super().__init__()
        
        self.n_hidden = n_hidden
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.75
        )
        
        self.classifier = nn.Linear(n_hidden, n_classes)
        
    def forward(self, x):
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        
        out = hidden[-1]
        return self.classifier(out)

In [48]:
class DrowsyPredictor(pl.LightningModule):
    
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.model = DrowsyModel(n_features, n_classes)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x, label=None):
        output = self.model(x)
        loss = 0
        if label is not None:
            loss = self.criterion(output, label)
        return loss, output
        
    def training_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("train_loss", loss, prog_bar=True, logger=True)
        self.log("train_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def validation_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.log("val_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def test_step(self, batch, batch_idx):
        sequences = batch["sequence"]
        labels = batch["label"]
        loss, outputs = self(sequences, labels)
        predictions = torch.argmax(outputs, dim=1)
        step_accuracy = accuracy(predictions, labels)
        
        self.log("test_loss", loss, prog_bar=True, logger=True)
        self.log("test_accuracy", step_accuracy, prog_bar=True, logger=True)
        
        return {"loss": loss, "accuracy": step_accuracy}
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.0001)

In [49]:
model = DrowsyPredictor(
    n_features=len(FEATURE_COLUMNS), 
    n_classes=len(label_encoder.classes_)
)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir ./lightning_logs --host 0.0.0.0 --port=8888

In [51]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint-corr-0",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

logger = TensorBoardLogger("lightning_logs", name="Drowsy")

trainer = pl.Trainer(
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    gpus=[1],
#     progress_bar_refresh_rate=20
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


## Training 

In [52]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DrowsyModel      | 1.4 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.429     Total estimated model params size (MB)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:623: UserWarning: Checkpoint directory /Workspace/code/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Epoch 0:  89%|████████▊ | 226/255 [02:39<00:20,  1.42it/s, loss=0.634, v_num=19, train_loss=0.564, train_accuracy=0.722]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 255/255 [02:49<00:00,  1.51it/s, loss=0.634, v_num=19, train_loss=0.564, train_accuracy=0.722, val_loss=0.595, val_accuracy=0.698]
                                                           

Epoch 0, global step 225: val_loss reached 0.59524 (best 0.59524), saving model to "/Workspace/code/checkpoints/best-checkpoint40x80-v3.ckpt" as top 1


Epoch 1:  89%|████████▊ | 226/255 [02:39<00:20,  1.41it/s, loss=0.549, v_num=19, train_loss=0.706, train_accuracy=0.722, val_loss=0.595, val_accuracy=0.698]
Validating: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 255/255 [02:51<00:00,  1.49it/s, loss=0.549, v_num=19, train_loss=0.706, train_accuracy=0.722, val_loss=0.599, val_accuracy=0.692]
                                                           

Epoch 1, global step 451: val_loss was not in top 1


...

Epoch 98:  89%|████████▊ | 226/255 [02:40<00:20,  1.41it/s, loss=0.0739, v_num=19, train_loss=0.0329, train_accuracy=1.000, val_loss=0.290, val_accuracy=0.908] 
Validating: 0it [00:00, ?it/s]
Epoch 98: 100%|██████████| 255/255 [02:50<00:00,  1.50it/s, loss=0.0739, v_num=19, train_loss=0.0329, train_accuracy=1.000, val_loss=0.282, val_accuracy=0.917]
                                                           

Epoch 98, global step 22373: val_loss was not in top 1


Epoch 99:  89%|████████▊ | 226/255 [02:39<00:20,  1.42it/s, loss=0.24, v_num=19, train_loss=0.328, train_accuracy=0.944, val_loss=0.282, val_accuracy=0.917]   
Validating: 0it [00:00, ?it/s]
Epoch 99: 100%|██████████| 255/255 [02:49<00:00,  1.50it/s, loss=0.24, v_num=19, train_loss=0.328, train_accuracy=0.944, val_loss=0.369, val_accuracy=0.893]
                                                           

Epoch 99, global step 22599: val_loss was not in top 1


Epoch 99: 100%|██████████| 255/255 [02:49<00:00,  1.50it/s, loss=0.24, v_num=19, train_loss=0.328, train_accuracy=0.944, val_loss=0.369, val_accuracy=0.893]


In [53]:
trainer.test(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 100%|██████████| 29/29 [00:10<00:00,  2.56it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': 0.8814762234687805, 'test_loss': 0.4126664102077484}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 29/29 [00:10<00:00,  2.78it/s]


[{'test_loss': 0.4126664102077484, 'test_accuracy': 0.8814762234687805}]

## Train from check point

In [ ]:
trainer.fit(model, data_module, ckpt_path="~/Workspace/code/checkpoints/best-checkpoint-corr-7-.ckpt")

## Predictions

In [55]:
trained_model = DrowsyPredictor.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path,
    n_features=len(FEATURE_COLUMNS),
    n_classes=len(label_encoder.classes_)
)

trained_model.freeze()

In [ ]:
from tqdm import tqdm

test_dataset = DrowsyDataset(test_sequences)

predictions = []
labels = [] 

for item in tqdm(test_dataset):
    sequence = item["sequence"]
    label = item["label"]
    
    _, output = trained_model(sequence.unsqueeze(dim=0))
    prediciton = torch.argmax(output, dim=1)
    predictions.append(prediciton.item())
    labels.append(label.item())

 88%|████████▊ | 1235/1409 [2:43:14<46:46, 16.13s/it] 

In [ ]:
print(
    classification_report(labels, predictions, target_names=label_encoder.classes_)
)

In [ ]:
import seaborn as sns
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha="right")
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha="right")
    plt.ylabel("True Drowsy")
    plt.xlabel("Predicted Drowsy")

In [ ]:
cm = confusion_matrix(labels, predictions)
df_cm = pd.DataFrame(
    cm, index=label_encoder.classes_, columns=label_encoder.classes_
)

In [ ]:
show_confusion_matrix(df_cm)